In [1]:
from keras.models import load_model

In [2]:
loaded_model = load_model('knn_file.h5')

In [3]:
import cv2

In [4]:
# %load mnist_gui.py
import tkinter as tk
from tkinter import *
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import cv2
import numpy as np
global d
class ImageGenerator:
    def __init__(self,parent,posx,posy,*kwargs):
        self.parent = parent
        self.posx = posx
        self.posy = posy
        self.sizex = 400
        self.sizey = 300
        self.b1 = "up"
        self.xold = None
        self.yold = None 
        tk.Label(self.parent,text="Predicting Hand-Written Digits",fg="green",font=("",15,"bold")).place(x=50,y=10)
        self.drawing_area=tk.Canvas(self.parent,width=self.sizex,height=self.sizey)
        self.drawing_area.place(x=self.posx,y=self.posy)
        self.drawing_area.bind("<Motion>", self.motion)
        self.drawing_area.bind("<ButtonPress-1>", self.b1down)
        self.drawing_area.bind("<ButtonRelease-1>", self.b1up)
        self.button=tk.Button(self.parent,text="Save Image",width=10,bg='white',command=self.save)
        self.button.place(x=self.sizex/5,y=self.sizey+75)
        self.button1=tk.Button(self.parent,text="Clear",width=10,bg='white',command=self.clear)
        self.button1.place(x=(self.sizex/5)+80,y=self.sizey+75)
        
        self.button2=tk.Button(self.parent,text="Predict",width=10,bg='white',command=self.predict)
        self.button2.place(x=(self.sizex/5)+160,y=self.sizey+75)
        
       
        
        self.image=Image.new("RGB",(400,300),(255,255,255))
        self.draw=ImageDraw.Draw(self.image)

    def save(self):
        filename = "temp.jpg"
        self.image.save(filename)

    def clear(self):
        self.drawing_area.delete("all")
        self.image=Image.new("RGB",(400,300),(255,255,255))
        self.draw=ImageDraw.Draw(self.image)

    def b1down(self,event):
        self.b1 = "down"

    def b1up(self,event):
        self.b1 = "up"
        self.xold = None
        self.yold = None

    def motion(self,event):
        if self.b1 == "down":
            if self.xold is not None and self.yold is not None:
                event.widget.create_line(self.xold,self.yold,event.x,event.y,smooth='true',width=10,fill='black')
                event.widget.create_oval(self.xold,self.yold,event.x,event.y,fill='black')
                self.draw.line(((self.xold,self.yold),(event.x,event.y)),(0,0,0),width=10)
                #self.draw.ellipse(((self.xold,self.yold),(event.x,event.y)),(0,0,0),fill='black')

        self.xold = event.x
        self.yold = event.y
        
    def predict(self):
        s=''
        
        img =cv2.imread('temp.jpg')
        #img_1 = np.invert(img)
        
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray_img,(7,7),0)
        
       

        ret,th_img = cv2.threshold(blur, 110, 255, cv2.THRESH_BINARY_INV)
        
       
        
        ctrs, hier = cv2.findContours(th_img, cv2.RETR_EXTERNAL, cv2.THRESH_BINARY_INV)
        
        for contour in ctrs:
            
            [x,y,w,h] = cv2.boundingRect(contour)
            #cv2.rectangle(img,(x-10,y-10),(x+(w+10),y+(h+10)),(0,0,0),2)
            
            roi = img[y-12:y+(h+12),x-12:x+(w+12)]
            
            roi = np.invert(roi)
            
            #cv2.imshow('object',roi)
            #kernel = np.zeros((35,35), np.uint8)

            #roi = cv2.erode(roi, kernel, iterations=1)
            
            #cv2.imshow('object',roi)
            roi = cv2.resize(roi, (28, 28))
            #cv2.imwrite('sample/'+str(i)+'.png',roi)
           
            
            arr = np.array(roi).astype(float)/255
            #print (arr.shape)
            arr = arr.reshape(1,28,28,3)
            re=loaded_model.predict(arr).argmax()
            cv2.putText(img, str(re), (x-8,y-8),cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 255), 3)
            #print(re)
            
            
        cv2.imwrite('output.png',img)
        
        cv2.imshow('final',img)
        #cv2.imshow('threshold',th_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
            
                
        

       
        
    
    
        

if __name__ == "__main__":
    root=tk.Tk()
    root.wm_geometry("%dx%d+%d+%d" % (550, 450, 10, 10))
    root.config(bg='white')
    ImageGenerator(root,60,60)
    root.mainloop()
